In [50]:
## Connect to DB server on AWS
import mysql.connector 
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = 'crime_nyc'
)
cursor = cnx.cursor()
cnx.database = 'crime_nyc'

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import itertools

In [52]:
query = ("SELECT * FROM crime2019;")
cursor.execute(query)
#cursor.close()
#cnx.close()

In [53]:
result = cursor.fetchall()

In [54]:
df = pd.DataFrame(result)
df.columns = [x[0] for x in cursor.description]

,index,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,...,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,zipcode,med_income
0,0,203078287,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,...,0,25-44,M,BLACK HISPANIC,990563,203120,40.724200,-73.977226,10009,59929.0
1,1,203072424,09/30/2019,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,Q,...,0,18-24,M,BLACK,1040611,190715,40.689974,-73.796769,11433,42887.0
2,2,203061215,09/30/2019,905.0,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U3,M,S,...,0,18-24,M,WHITE,962989,160112,40.606129,-74.076570,10305,70758.0
3,3,203061218,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601502,F,S,...,0,25-44,M,WHITE HISPANIC,962822,174282,40.645023,-74.077217,10301,56848.0
4,4,203063729,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,Q,...,0,18-24,M,ASIAN / PACIFIC ISLANDER,1035521,188624,40.684266,-73.815138,11420,59832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167960,167960,191717230,01/01/2019,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",351.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450001,M,Q,...,0,25-44,M,WHITE HISPANIC,1013749,204160,40.727008,-73.893572,11379,69843.0
167961,167961,191717236,01/01/2019,744.0,BAIL JUMPING 3,359.0,OFFENSES AGAINST PUBLIC ADMINI,PL 2155500,M,Q,...,0,25-44,M,WHITE HISPANIC,1032428,198872,40.712411,-73.826217,11415,63549.0
167962,167962,191717241,01/01/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,S,...,0,25-44,F,BLACK,952218,173159,40.641908,-74.115423,10310,61925.0
167963,167963,191719207,01/01/2019,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200502,F,B,...,2,25-44,F,WHITE HISPANIC,1006918,238567,40.821466,-73.918101,10451,26754.0


In [48]:
make_df(cursor,query)

InternalError: Unread result found

In [ ]:
# from sqlalchemy import create_engine
# engine = create_engine('mysql://serverandres.c43rrwosnqfo.us-east-2.rds.amazonaws.com')  # Creating the engine
# query = "SELECT * FROM crime2019"  # String containing the SQL query

# df = pd.read_sql_query(query, engine, index_col="index")  # Using the first column as indices
# df.head()